In [2]:
import pandas as pd
import numpy as np
import pandas_profiling as pf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
custDemo = pd.read_excel(r'D:\AnalytxLabs\Machine Learning Projects\11. Capstone Case Study - Predict Cred Card Consumption\CustomerDemographics.xlsx')
custDemo.head()

,ID,account_type,gender,age,Income,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction
0,19427,current,M,63,MEDIUM,30.1,10,628.0,1,5.0
1,16150,current,M,36,MEDIUM,14.4,10,656.0,0,12.0
2,11749,current,F,28,MEDIUM,4.8,10,314.0,1,13.0
3,11635,current,M,32,MEDIUM,9.6,2,614.0,1,19.0
4,8908,current,M,32,HIGH,12.0,7,750.0,1,18.0


In [4]:
custBehaviour = pd.read_excel(r'D:\AnalytxLabs\Machine Learning Projects\11. Capstone Case Study - Predict Cred Card Consumption\CustomerBehaviorData.xlsx')
custBehaviour.head()

,ID,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
0,17051,3412.60,2909.34,1077.36,3084.50,4615.97,6693.0,19.0,10,2,...,1,108,97916.0,45253.64,47635.00,2,18,19947.0,Y,3448.84
1,11491,18133.76,3877.00,6123.78,5597.43,64620.00,6117.0,4.0,60,1,...,1,4,25125.0,12912.33,18369.00,3,49,20585.0,Y,3812.69
2,7433,6363.65,735.49,8799.00,13768.00,38266.00,2638.0,2.0,83,47,...,70,11,38546.0,93567.51,26713.00,47,49,13950.0,Y,9432.90
3,14606,12765.66,4429.16,16745.86,6360.00,29063.97,3711.0,12.0,2,16,...,43,1,180315.0,176394.48,35062.74,3,37,37018.0,Y,144.61
4,8381,27819.70,1944.00,7006.50,2228.50,1096.25,1065.0,15.0,13,61,...,2,14,13480.0,3167.00,19056.00,2,15,24274.0,Y,1887.89


In [5]:
credConsumption = pd.read_excel(r'D:\AnalytxLabs\Machine Learning Projects\11. Capstone Case Study - Predict Cred Card Consumption\CreditConsumptionData.xlsx')
credConsumption.head()

,ID,cc_cons
0,12554,20014.0
1,17645,10173.0
2,7604,16095.0
3,1590,7707.0
4,16556,96408.0


In [6]:
cust_demo_behaviour = pd.merge(left= custDemo, right = custBehaviour, on= 'ID', how= 'left')

In [7]:
cust_cred_consumption = pd.merge(left= cust_demo_behaviour, right = credConsumption, on= 'ID', how= 'left')

In [8]:
sum(cust_cred_consumption.duplicated())

0

In [9]:
cust_cred_consumption.isna().sum()

ID                                 0
account_type                       1
gender                             1
age                                0
Income                             1
Emp_Tenure_Years                   0
Tenure_with_Bank                   0
region_code                        1
NetBanking_Flag                    0
Avg_days_between_transaction       3
cc_cons_apr                        0
dc_cons_apr                        0
cc_cons_may                        1
dc_cons_may                        0
cc_cons_jun                        0
dc_cons_jun                        1
cc_count_apr                       1
cc_count_may                       0
cc_count_jun                       0
dc_count_apr                       0
dc_count_may                       0
dc_count_jun                       0
card_lim                           0
personal_loan_active               0
vehicle_loan_active                0
personal_loan_closed               1
vehicle_loan_closed                0
i

In [10]:
cust_cred_consumption.ID.value_counts().count()

20000

In [11]:
report = pf.ProfileReport(cust_cred_consumption)

In [ ]:
report.to_file('report.html')

In [13]:
custCred_existing = cust_cred_consumption[cust_cred_consumption.cc_cons.isnull() == 0]
custCred_new = cust_cred_consumption[cust_cred_consumption.cc_cons.isnull() == 1]

In [14]:
custCred_existing.shape

(15000, 49)

In [15]:
custCred_new.shape

(5000, 49)

In [16]:
custCred_existing.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 111 to 15110
Data columns (total 49 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   ID                            15000 non-null  int64  
 1   account_type                  15000 non-null  object 
 2   gender                        15000 non-null  object 
 3   age                           15000 non-null  int64  
 4   Income                        15000 non-null  object 
 5   Emp_Tenure_Years              15000 non-null  float64
 6   Tenure_with_Bank              15000 non-null  int64  
 7   region_code                   15000 non-null  float64
 8   NetBanking_Flag               15000 non-null  int64  
 9   Avg_days_between_transaction  14999 non-null  float64
 10  cc_cons_apr                   15000 non-null  float64
 11  dc_cons_apr                   15000 non-null  float64
 12  cc_cons_may                   15000 non-null  float64
 13 

In [17]:
def missing_value(x):
    if x.dtype == 'object':
        x = x.fillna(x.mode()[0])
    else:
        x = x.fillna(x.median())
    return x

In [19]:
def outliers_treatment(x):
    if (x.dtype == 'float64') or (x.dtype == 'int64'):
        x = x.clip(lower = x.quantile(0.01), upper = x.quantile(0.99))
    else:
        x
    return x

In [20]:
custCred_existing = custCred_existing.apply(missing_value)
custCred_existing = custCred_existing.apply(outliers_treatment)

In [21]:
custCred_existing = pd.get_dummies(custCred_existing,columns=['account_type','gender','Income','loan_enq'],drop_first= True)

In [22]:
custCred_existing.cc_cons.plot(kind = 'hist', bins = 10)

<AxesSubplot:xlabel='investment_1', ylabel='Frequency'>

In [33]:
sns.distplot(custCred_existing.cc_cons)
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
<ipython-input-33-bd43d669f110>:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [24]:
sns.distplot(np.log(custCred_existing.cc_cons))
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\seaborn\distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
<ipython-input-24-1ec6fe5c90e1>:2: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [25]:
custCred_existing['ln_cc_cons'] = np.log(custCred_existing.cc_cons)

In [26]:
custCred_existing.head()

,ID,age,Emp_Tenure_Years,Tenure_with_Bank,region_code,NetBanking_Flag,Avg_days_between_transaction,cc_cons_apr,dc_cons_apr,cc_cons_may,...,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,cc_cons,account_type_saving,gender_M,Income_LOW,Income_MEDIUM,ln_cc_cons
111,12554.0,35,15.0,1,708.0,0,17.0,24893.00,378.00,10288.00,...,2,65,50836.0,1674.09,20014.0,0,1,0,1,9.904187
112,17645.0,35,15.0,6,233.0,0,14.0,18941.62,966.00,20672.00,...,21,63,23226.0,13043.34,10173.0,0,1,0,0,9.227492
113,7604.0,55,24.5,1,802.0,1,3.0,5678.87,2724.00,1964.50,...,7,1,27036.0,25375.27,16095.0,0,1,0,1,9.686264
114,1590.0,29,4.5,6,867.0,1,4.0,30489.50,1236.00,12609.88,...,21,83,43037.0,3544.33,7707.0,0,0,0,1,8.949884
115,16556.0,34,12.6,4,802.0,1,7.0,7441.40,6906.04,4364.00,...,8,32,32044.0,12780.44,96408.0,0,1,0,1,11.476344


In [28]:
corr = custCred_existing.corrwith(custCred_existing.ln_cc_cons)

In [30]:
corr.to_csv('corr.csv')

In [39]:
corr[np.abs(corr > 0.05)]

cc_cons       0.825807
ln_cc_cons    1.000000
dtype: float64